<a href="https://colab.research.google.com/github/sarveshraghavan/RAP/blob/main/NewsChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- 1. Install Dependencies ---
!pip install transformers accelerate torch gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.5 MB/s eta 0:00:00


In [4]:


# --- 1. Import Libraries ---
import os
import requests
import re
from typing import List
import gradio as gr
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# --- 2. News API Settings ---
SERP_API_KEY = "6d54298cef76d40c1df8df9c1aa84f50ea72d44a96c31c1796e157f1b8a4d2c6"  # Replace with your actual key

# --- 3. Load Summarization Agent (TinyLlama) ---
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
summarizer = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=150)

# --- 4. Summarization Function ---
def summarize_news(text, style="Formal"):
    prompt_map = {
        "Formal": f"Summarize this news in a professional tone:\n{text}",
        "Casual": f"Hey! Can you explain this news casually?\n{text}",
        "Bullet Points": f"Give 3 bullet point summary for:\n{text}"
    }
    prompt = prompt_map.get(style, prompt_map["Formal"])
    response = summarizer(prompt, do_sample=True, temperature=0.7)[0]['generated_text']
    return response.split(prompt)[-1].strip()

# --- 5. Greeting Detection ---
def is_greeting(text):
    greetings = ["hi", "hello", "hey", "hola", "namaste"]
    return text.lower().strip() in greetings

# --- 6. Extract Topic ---
def extract_topic(query):
    query = query.lower()
    keywords = ["news on", "about", "regarding", "get me", "fetch me", "fetch", "news", "for"]
    for k in keywords:
        query = query.replace(k, "")
    return query.strip()

# --- 7. Fetch News ---
def fetch_news(topic):
    url = f"https://serpapi.com/search.json?q={topic}&tbm=nws&api_key={SERP_API_KEY}"
    res = requests.get(url).json()
    news_results = res.get("news_results", [])
    if not news_results:
        return f"No news found for '{topic}'", ""
    return news_results

# --- 8. Main Handler Function ---
def search_and_summarize(query, style):
    if is_greeting(query):
        return f"<p style='font-size: 18px;'>👋 Hello there! How can I help you today?</p>"

    topic = extract_topic(query)
    news_list = fetch_news(topic)

    if isinstance(news_list, str):  # Error message
        return news_list

    display_blocks = ""
    for item in news_list[:3]:
        if isinstance(item, dict):
            title = item.get("title", "No Title")
            link = item.get("link", "#")
            content = item.get("snippet", "No summary available")
            summary = summarize_news(content, style)

            display_blocks += f"""
            <div style='padding:10px; border:1px solid #ccc; border-radius:10px; margin-bottom:15px'>
                <h3><a href="{link}" target="_blank">{title}</a></h3>
                <p><b> Summary:</b><br>{summary}</p>
            </div>
            """
    if not display_blocks:
        return "⚠️ No news articles could be parsed."
    return display_blocks

# --- 9. Gradio Interface ---
with gr.Blocks() as demo:
    gr.Markdown("## 🧠 AI News Summarizer")

    with gr.Row():
        query = gr.Textbox(label="Enter your query", placeholder="e.g., Fetch me news on Google")
        style = gr.Dropdown(choices=["Formal", "Casual", "Bullet Points"], label="Summary Style", value="Formal")

    btn = gr.Button("Fetch & Summarize")
    output = gr.HTML()

    btn.click(fn=search_and_summarize, inputs=[query, style], outputs=output)

demo.launch(debug=True)


Device set to use cuda:0


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://3ff78815379daff90b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://3ff78815379daff90b.gradio.live


In [5]:
!pip install langgraph langchain langchainhub --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.5/152.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 19.4 MB/s eta 0:00:00


In [8]:
!pip install langchain_community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00


In [11]:
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.runnables import Runnable
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
from langchain.memory import ConversationBufferMemory
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline

# ---- Step 1: Load TinyLlama model ----
tiny_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=150)
llm = HuggingFacePipeline(pipeline=tiny_pipeline)

# ---- Step 2: Conversation Memory ----
memory = ConversationBufferMemory(return_messages=True)

# ---- Step 3: Agent State ----
class AgentState(TypedDict):
    messages: Annotated[list, memory]

# ---- Step 4: Context Extractor ----
def extract_context(messages):
    context = ""
    for msg in messages:
        if isinstance(msg, HumanMessage):
            context += f"User: {msg.content}\n"
        elif isinstance(msg, AIMessage):
            context += f"AI: {msg.content}\n"
    return context

# ---- Step 5: Respond Agent ----
def respond_agent(state: AgentState) -> AgentState:
    history = state["messages"]
    context_str = extract_context(history)

    prompt = f"""You are a helpful news analyst. Based on the previous summary and discussion, answer the follow-up question accurately.

Here is the conversation history so far:
{context_str}

Now answer the latest user question based on the context above.
Answer:"""

    response = llm.invoke(prompt)
    # In case response is a list/dict structure (some HF pipelines behave this way)
    if isinstance(response, list) and 'generated_text' in response[0]:
        response_text = response[0]['generated_text']
    else:
        response_text = str(response)

    return {"messages": history + [AIMessage(content=response_text)]}


Device set to use cuda:0


In [13]:
builder = StateGraph(AgentState)
builder.add_node("agent", respond_agent)
builder.set_entry_point("agent")
builder.set_finish_point("agent")

# ✅ Compile graph
graph = builder.compile()

In [15]:
# Global var to hold convo history
convo_context = []

# Extend UI
with gr.Blocks() as demo:
    gr.Markdown("## 🧠 AI News Summarizer")

    with gr.Row():
        topic = gr.Textbox(label="Enter a Topic", placeholder="e.g., AI, Apple, Olympics")
        style = gr.Dropdown(choices=["Formal", "Casual", "Bullet Points"], label="Summary Style", value="Formal")

    btn = gr.Button("Fetch & Summarize")
    output = gr.HTML()


    btn.click(fn=search_and_summarize, inputs=[topic, style], outputs=output)


demo.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://adafea0b58fa75d2f2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://adafea0b58fa75d2f2.gradio.live


  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=3619c0c9fccf5df2cf498ca9ba23b9ac32d8b1c4c84be73331543d5420044774
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results
